# 3. Neural Style Transfer on AKS

We've tested locally in the previous notebook. Now use an AKS cluster and test that our neural style transfer script still works as expected when running across multiple nodes in parallel on AKS.

1. Build AKS Docker Image
2. Test style transfer on Docker locally
3. Push docker image to Docker hub
4. Provision AKS cluster 
5. Test style transfer on parallel on AKS cluster

---

### Import packages and load .env

In [1]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import subprocess
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Build AKS Docker Image

In [89]:
%%writefile aks/requirements.txt
azure==4.0.0
torch==0.4.1
torchvision==0.2.1

Overwriting aks/requirements.txt


In [90]:
%%writefile aks/Dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        curl \
        git \
        nginx \
        supervisor \
        wget && \
        rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.6
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION && \
    /opt/conda/bin/conda clean -ya
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
ENV PYTHONPATH /code/:$PYTHONPATH

RUN mkdir /app
WORKDIR /app
ADD process_images_from_queue.py /app
ADD style_transfer.py /app
ADD main.py /app
ADD util.py /app
ADD requirements.txt /app

RUN pip install --no-cache-dir -r requirements.txt

CMD ["python", "main.py"]

Overwriting aks/Dockerfile


In [91]:
!sudo docker build -t {get_key(env_path, "AKS_IMAGE")} aks

Sending build context to Docker daemon  68.61kB
Step 1/17 : FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
 ---> f4f6aaaaa057
Step 2/17 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Using cache
 ---> 4196af2ba86e
Step 3/17 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates         cmake         curl         git         nginx         supervisor         wget &&         rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 8ddcde9d280a
Step 4/17 : ENV PYTHON_VERSION=3.6
 ---> Using cache
 ---> 5a047de1f83a
Step 5/17 : RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  &&     chmod +x ~/miniconda.sh &&     ~/miniconda.sh -b -p /opt/conda &&     rm ~/miniconda.sh &&     /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION &&     /opt/conda/bin/conda c

  Running setup.py install for pycparser: started
    Running setup.py install for pycparser: finished with status 'done'


Removing intermediate container 7f6852174a46
 ---> d8f41550c481
Step 17/17 : CMD ["python", "main.py"]
 ---> Running in c4251769a49d
Removing intermediate container c4251769a49d
 ---> 0adaa5640104
Successfully built 0adaa5640104
Successfully tagged batchscoringdl_aks_app:latest


Tag and push docker image

In [92]:
repo = "{}/{}".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "AKS_IMAGE"))

In [93]:
!sudo docker tag {get_key(env_path, "AKS_IMAGE")} {repo}

In [94]:
!sudo docker push {repo}

The push refers to repository [docker.io/jiata/batchscoringdl_aks_app]

72857cc6: Preparing 
fa001336: Preparing 
04627bec: Preparing 
d0ba3263: Preparing 
be141c1f: Preparing 
79241778: Preparing 
8452f77e: Preparing 
aad0d176: Preparing 
ff05626e: Preparing 
9048222b: Preparing 
f7dc85a1: Preparing 
2df89268: Preparing 
d8f0884d: Preparing 
87fdb58c: Preparing 
8fb03d12: Preparing 
843615e2: Preparing 
452f77e: Waiting g 
9c0f8a0b: Preparing 
ad0d176: Waiting g 
2857cc6: Pushed   1.298GB/1.267GBPushing  1.158GB/1.267GBlatest: digest: sha256:9363fc51bf651300fd9c3cbb98a80783529495ceeb23de96c060ba9c48984bdb size: 4507


### Create Docker image to run in ACI

Create our Dockerfile and save it to the directory, `/aci`.

In [ ]:
%%writefile aci/Dockerfile

FROM continuumio/miniconda3

RUN mkdir /app
WORKDIR /app
ADD add_images_to_queue.py /app
ADD preprocess.py /app
ADD postprocess.py /app
ADD util.py /app
ADD main.py /app

# RUN conda install -c anaconda ffmpeg
RUN conda install -c conda-forge -y ffmpeg
RUN pip install azure
RUN pip install flask

CMD ["python", "main.py"]

Build the Docker image

In [ ]:
!sudo docker build -t {get_key(env_path, "ACI_IMAGE")} aci

Tag and push.

In [ ]:
repo = "{}/{}".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "ACI_IMAGE"))

In [ ]:
!sudo docker tag {get_key(env_path, "ACI_IMAGE")} {repo}

In [ ]:
!sudo docker push {repo}

### Provision AKS cluster and set up blobfuse

Set how many nodes you want to provision.

In [8]:
node_count = 5

Check that there are enough core of the "Standard_NC6s_v3". If not, check that there are enough core of the "Standard_D2s_v3". If not, raise exception. 

In [9]:
vm_dict = {
    "NCSv3": {
        "size": "Standard_NC6s_v3",
        "cores": 6
    },
    "DSv3": {
        "size": "Standard_D2s_v3",
        "cores": 2
    }
}

print("Checking quota for family size NCSv3...")
vm_family = "NCSv3"
requested_cores = node_count * vm_dict[vm_family]["cores"]

def check_quota(vm_family):
    """
    returns quota object
    """
    results = subprocess.run([
        "az", "vm", "list-usage", 
        "--location", get_key(env_path, "REGION"), 
        "--query", "[?contains(localName, '%s')].{max:limit, current:currentValue}" % (vm_family)
    ], stdout=subprocess.PIPE)
    quota = json.loads(''.join(results.stdout.decode('utf-8')))
    return int(quota[0]['max']) - int(quota[0]['current'])

diff = check_quota(vm_family)
if diff <= requested_cores:
    print("Not enough cores of NCSv3 in region, asking for {} but have {}".format(requested_cores, diff))
    
    print("Retrying with family size DSv3...")
    vm_family = "DSv3"
    requested_cores = node_count * vm_dict[vm_family]["cores"]
    
    diff = check_quota(vm_family)
    if diff <= requested_cores:
        print("Not enough cores of DSv3 in region, asking for {} but have {}".format(requested_cores, diff))
        raise Exception("Core Limit", "Note enough cores to satisfy request")

print("There are enough cores, you may continue...") 

Checking quota for family size NCSv3...
There are enough cores, you may continue...


Create the aks cluster. This step may take a while... Please note that this step creates another resource group in your subscription containing the actual compute of the AKS cluster.

In [10]:
!az aks create \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --name {get_key(env_path, "AKS_CLUSTER")} \
    --node-count {node_count} \
    --node-vm-size {vm_dict[vm_family]["size"]} \
    --generate-ssh-keys \
    --service-principal {get_key(env_path, "SP_CLIENT")} \
    --client-secret {get_key(env_path, "SP_SECRET")}

{- Finished ..
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "count": 5,
      "maxPods": 110,
      "name": "nodepool1",
      "osDiskSizeGb": 30,
      "osType": "Linux",
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6s_v3",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "batchscori-jiataaks04-edf507",
  "enableRbac": true,
  "fqdn": "batchscori-jiataaks04-edf507-91fe0bd4.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/jiataaks04/providers/Microsoft.ContainerService/managedClusters/batchscoringdlcluster",
  "kubernetesVersion": "1.9.11",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQD1bKwjkSweMxAgRToml+F++4nnvf/kUaKQ3GukcUZPdsb12ZklDUEJZLyp/zUCuIQhVKU5YC/+SekDx2gMjmnlPKlDKYEJm/t1EUoSBSQD4Axmfrbkyuen8kD0O/c4U9ysVVuyD9V23DYxbcAxs/y/KkAHL1P8FOz7Twg78l/WuF9rRXRC

Install Kubectl - this tool is used to manage the kubernetes cluster.

In [11]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


In [12]:
!az aks get-credentials \
    --resource-group {get_key(env_path, 'RESOURCE_GROUP')}\
    --name {get_key(env_path, 'AKS_CLUSTER')}

Merged "batchscoringdlcluster" as current context in /home/jiata/.kube/config


Check that our nodes are up and ready.

In [13]:
!kubectl get nodes

NAME                       STATUS   ROLES   AGE     VERSION
aks-nodepool1-51750795-0   Ready    agent   3h53m   v1.9.11
aks-nodepool1-51750795-1   Ready    agent   3h53m   v1.9.11
aks-nodepool1-51750795-2   Ready    agent   3h53m   v1.9.11
aks-nodepool1-51750795-3   Ready    agent   3h52m   v1.9.11
aks-nodepool1-51750795-4   Ready    agent   3h53m   v1.9.11


Now we setup our AKS cluster so that we have blob storage mounted onto the nodes using blob fuse. More info [here](https://github.com/Azure/kubernetes-volume-drivers/tree/master/flexvolume/blobfuse).

Install blobfuse driver on every agent VM.

In [14]:
!kubectl create -f https://raw.githubusercontent.com/Azure/kubernetes-volume-drivers/master/flexvolume/blobfuse/deployment/blobfuse-flexvol-installer-1.9.yaml

Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/Azure/kubernetes-volume-drivers/master/flexvolume/blobfuse/deployment/blobfuse-flexvol-installer-1.9.yaml": namespaces "flex" already exists
Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/Azure/kubernetes-volume-drivers/master/flexvolume/blobfuse/deployment/blobfuse-flexvol-installer-1.9.yaml": daemonsets.apps "blobfuse-flexvol-installer" already exists


Check daemonset status.

In [15]:
!watch kubectl describe daemonset blobfuse-flexvol-installer --namespace=flex
!watch kubectl get po --namespace=flex -o wide

>bfuse-flexvol-installer-m2sp9   1/1     Running   03h7m   10.244.0.4   aks-nodepool1-51750795-4   <none><none>ume/blobfuse-flexvolumePort:<none>Host Port:    <none>Environment:  <none>Mounts:/etc/kubernetes/volumeplugins/ from volplugins (rw)/var/log/ from varlog (rw)38)07Every 2.0s: kubectl get po --names...  jiata-ubuntu-ws: Tue Dec 11 19:33:34 2018NAMEREADY   STATUS    RESTARTS   AGE    IPNODENOMINATED NODE   READINESS GATES2   aks-nodepool1-51750795-3   <none><none>68

Set up credentials for blobfuse.

In [16]:
!kubectl create secret generic blobfusecreds \
    --from-literal accountname={get_key(env_path, 'STORAGE_ACCOUNT_NAME')} \
    --from-literal accountkey={get_key(env_path, 'STORAGE_ACCOUNT_KEY')} \
    --type="azure/blobfuse"

Error from server (AlreadyExists): secrets "blobfusecreds" already exists


Set the mount directory on our AKS cluster.

In [85]:
mount_dir = "/data"

### Create our AKS deployments

We need to deploy both our aci and aks docker images to the AKS cluster. Since we'll need to set up our gpu and drivers and blobfuse mount point for both deployments, we'll set these up first:

In [95]:
volume_mounts = [
    {"name": "nvidia", "mountPath": "/usr/local/nvidia"},
    {"name": "blob", "mountPath": mount_dir},
]

resources = {
    "requests": {"alpha.kubernetes.io/nvidia-gpu": 1},
    "limits": {"alpha.kubernetes.io/nvidia-gpu": 1},
}

volumes = [
    {"name": "nvidia", "hostPath": {"path": "/usr/local/nvidia"}},
    {
        "name": "blob",
        "flexVolume": {
            "driver": "azure/blobfuse",
            "readOnly": False,
            "secretRef": {"name": "blobfusecreds"},
            "options": {
                "container": get_key(env_path, "STORAGE_CONTAINER_NAME"),
                "tmppath": "/tmp/blobfuse",
                "mountoptions": "--file-cache-timeout-in-seconds=120 --use-https=true",
            },
        },
    },
]

env = [
    {
        "name": "MOUNT_DIR", 
        "value": mount_dir
    },
    {
        "name": "LB_LIBRARY_PATH",
        "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib",
    },
    {
        "name": "DP_DISABLE_HEALTHCHECKS", 
        "value": "xids"
    },
    {
        "name": "STORAGE_MODEL_DIR",
        "value": get_key(env_path, "STORAGE_MODEL_DIR")
    },
    {
        "name": "SUBSCRIPTION_ID",
        "value": get_key(env_path, "SUBSCRIPTION_ID")
    },
    {
        "name": "RESOURCE_GROUP",
        "value": get_key(env_path, "RESOURCE_GROUP")
    },
    {
        "name": "REGION",
        "value": get_key(env_path, "REGION")
    },
    {
        "name": "SB_SHARED_ACCESS_KEY_NAME",
        "value": get_key(env_path, "SB_SHARED_ACCESS_KEY_NAME")
    },
    {
        "name": "SB_SHARED_ACCESS_KEY_VALUE",
        "value": get_key(env_path, "SB_SHARED_ACCESS_KEY_VALUE")
    },
    {
        "name": "SB_NAMESPACE",
        "value": get_key(env_path, "SB_NAMESPACE")
    },
    {
        "name": "SB_QUEUE", 
        "value": get_key(env_path, "SB_QUEUE")
    },
]

Define the aks deployment and save it to a `aks_deployment.json` file using the variables set above.

In [96]:
aks_deployment_json = {
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "aks-app", 
        "labels": {
            "purpose": "dequeue_messages_and_apply_style_transfer"
        }
    },
    "spec": {
        "replicas": node_count - 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "aks-app"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "aks-app",
                        "image": "{}/{}:latest".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "AKS_IMAGE")),
                        "volumeMounts": volume_mounts,
                        "resources": resources,
                        "ports": [{
                            "containerPort": 433
                        }],
                        "env": env,
                    }
                ],
                "volumes": volumes
            },
        },
    },
}

with open("aks_deployment.json", "w") as outfile:
    json.dump(aks_deployment_json, outfile, indent=4, sort_keys=True)
    outfile.write('\n\n')

Using the `aks_deployment.json` we created, create our deployment on AKS. This can take a few minutes...

In [98]:
!kubectl create -f aks_deployment.json

deployment.apps/aks-app created


Define the aci deployment and save it to a `aci_deployment.json` file using the variables set above.

In [97]:
aci_deployment_json = {
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "aci-app", 
        "labels": {
            "purpose": "pre_and_post_processing_and_queue_images"
        }
    },
    "spec": {
        "replicas":  1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "aci-app"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "aci-app",
                        "image": "{}/{}:latest".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "ACI_IMAGE")),
                        "volumeMounts": volume_mounts,
                        "resources": resources,
                        "ports": [{
                            "containerPort": 8080
                        }],
                        "env": env,
                    }
                ],
                "volumes": volumes
            },
        },
    },
}

with open("aci_deployment.json", "w") as outfile:
    json.dump(aci_deployment_json, outfile, indent=4, sort_keys=True)
    outfile.write('\n\n')

Using the `aci_deployment.json` we created, create our ffmpeg + queuer deployment on AKS. This can take a few minutes...

In [68]:
!kubectl create -f aci_deployment.json

deployment.apps/aci-app created


Expose the aci-app in the kubernetes cluster. This will open a public endpoint.

In [72]:
!kubectl expose deployment aci-app --type="LoadBalancer"

service/aci-app exposed


Run `!watch kubectl get services` and wait until the external ip goes from pending to being realized.

In [105]:
external_ip = !kubectl get services -o=jsonpath={.items[*].status.loadBalancer.ingress[0].ip}

To test that the deployments worked, lets hit the ip address and run a job with a new video file.

In [99]:
new_video_name = "aks_test_orangutan.mp4"

In [100]:
!cp data/orangutan.mp4 data/{new_video_name}

In [106]:
!curl {external_ip}":8080/process/"{new_video_name}

Processing aks_test_orangutan.mp4 in background...


### Basic Kubectl usage
You can use kubectl to perform basic monitoring. Use the following commands:
```bash
# monitor pods
!kubectl get pods

# print logs from a pod (<pod-name> can be found when calling 'get pods')
!kubectl logs <pod-name>

# check all services running on the cluster
!kubectl get services
```

### Monitor in kubernetes dashboard
You can use the Kubernetes dashboard to monitor the cluster using the following commands:

```bash
# use the kube_dashboard_access.yaml to create a deployment
!kubectl create -f kube_dashboard_access.yaml

# use this command to browse
!az aks browse -n {get_key(env_path, "AKS_CLUSTER")} -g {get_key(env_path, "RESOURCE_GROUP")}
```

### Additional commands for AKS

Scale your AKS cluster:

```bash 
!az aks scale \
    --name {get_key(env_path, "AKS_CLUSTER")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --node-count 10
```

Scale your deployment:
```bash
!kubectl scale deployment.apps/aks-app --replicas=10
```

---

Continue to the next [notebook](/notebooks/04_deploy_logic_app.ipynb).